## Initialization

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
import itertools
palette = itertools.cycle(sns.color_palette())
from func import get_data
# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Import Data

In [2]:
# Read the CSV file into a pandas DataFrame and display the first few rows
path_ella = 'C:/Users/ellad/Desktop/MB_ML/Projekt/train_heart.csv'
path_train = './train_heart.csv'
path_test = './test_heart.csv'

col = ['sex','cp','fbs','restecg','exang','slope','ca','thal']

[X_train,y_train,X_test,y_test] = get_data(path_train,path_test,col)


print("Training Dataset size:", len(X_train))
print("Test Dataset size:", len(X_test))

print('Training data shape: ',X_train.shape)
print('Test data shape: ',X_test.shape)

Training Dataset size: 1025
Test Dataset size: 303
Training data shape:  (1025, 30)
Test data shape:  (303, 30)


## NN Model

In [3]:
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
class FFNN(PyroModule):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN, self).__init__()
        
        # Architecture
        self.in_layer = PyroModule[nn.Linear](n_in, n_hidden)
        self.in_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_in]).to_event(2))

        self.h_layer = PyroModule[nn.Linear](n_hidden, n_hidden)
        self.h_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_hidden]).to_event(2))

        self.out_layer = PyroModule[nn.Linear](n_hidden, n_out)
        self.out_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_out, n_hidden]).to_event(2))

        # Activation functions
        self.tanh = nn.Tanh()
        
    def forward(self, X, y=None):
        X = self.tanh(self.in_layer(X))
        X = self.tanh(self.h_layer(X))
        X = self.out_layer(X)
        prediction_mean = X.squeeze(-1)

        """
        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Categorical(logits=alpha + X.matmul(beta)), obs=y)
        """

        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Bernoulli(logits=prediction_mean), obs=y)
            
        return y

In [4]:
X = torch.tensor(X_train.astype('float')).float()
y = torch.tensor(y_train).float()


In [5]:
# Define guide function
model = FFNN(n_in=X.shape[1], n_hidden=4, n_out=1)
guide = AutoDiagonalNormal(model)
pyro.clear_param_store()

In [6]:
# Define the number of optimization steps
n_steps = 20000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = Adam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X, y)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 1000.9
[1000] ELBO: 413.5
[2000] ELBO: 380.1
[3000] ELBO: 362.1
[4000] ELBO: 362.2
[5000] ELBO: 340.5
[6000] ELBO: 336.8
[7000] ELBO: 354.6
[8000] ELBO: 357.6
[9000] ELBO: 340.1
[10000] ELBO: 327.3
[11000] ELBO: 349.4
[12000] ELBO: 351.1
[13000] ELBO: 380.2
[14000] ELBO: 352.0
[15000] ELBO: 360.1
[16000] ELBO: 333.9
[17000] ELBO: 368.5
[18000] ELBO: 375.1
[19000] ELBO: 326.3


In [9]:
# Prepare test data for Pyro
X_test_tensor = torch.tensor(X_test.astype('float')).float()

from pyro.infer import Predictive
predictive = Predictive(model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test_tensor)

samples = samples['obs'].detach().squeeze()
y_hat = np.round(samples.mean(axis=0).numpy())

mae = np.mean(np.abs(y_test - y_hat))
print("MAE:", mae)
print("Accuracy:", 1.0*np.sum((y_hat) == y_test) / len(y_test))


AttributeError: 'Tensor' object has no attribute 'astype'